In [ ]:
!pip install imgaug
!pip install torch

In [2]:
import sys
sys.path.append('/kaggle/input/anti-noise-fgvr-v2')
sys.path.append('/kaggle/input/anti-noise-fgvr-v2/example')

In [3]:
import torch  

if torch.cuda.is_available():  
    print("CUDA is available!")  
    print(f"Number of GPUs: {torch.cuda.device_count()}")  
    print(f"Current GPU device: {torch.cuda.current_device()}")  
    print(f"Current GPU name: {torch.cuda.get_device_name(torch.cuda.current_device())}")  
else:  
    print("CUDA is not available.")

print(torch.cuda.device_count())
print(torch.cuda.is_available())

CUDA is available!
Number of GPUs: 1
Current GPU device: 0
Current GPU name: Tesla P100-PCIE-16GB
1
True


In [4]:
from __future__ import print_function
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import torchvision.models
from sam import SAM
from torch.utils.model_zoo import load_url as load_state_dict_from_url
import imgaug as ia
import imgaug.augmenters as iaa
from vic.loss import CharbonnierLoss
import numpy as np
import torchvision
from torch.autograd import Variable
from torchvision import transforms
from basic_conv import *
from model.smooth_cross_entropy import smooth_crossentropy
from utility.bypass_bn import enable_running_stats, disable_running_stats

def cosine_anneal_schedule(t, nb_epoch, lr):
    cos_inner = np.pi * (t % (nb_epoch))
    cos_inner /= (nb_epoch)
    cos_out = np.cos(cos_inner) + 1

    return float(lr / 2 * cos_out)

def test(net, criterion, batch_size, test_path):
    net.eval()
    use_cuda = torch.cuda.is_available()
    test_loss = 0
    correct = 0
    correct_com = 0
    total = 0
    idx = 0
    device = torch.device("cuda")

    transform_test = transforms.Compose([
        transforms.Resize((550, 550)),
        transforms.CenterCrop(448),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    testset = torchvision.datasets.ImageFolder(root=test_path,
                                               transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=4)

    for batch_idx, (inputs, targets) in enumerate(testloader):
        idx = batch_idx
        if use_cuda:
            inputs, targets = inputs.to(device), targets.to(device)
        inputs, targets = Variable(inputs, volatile=True), Variable(targets)
        output_1, output_2, output_3, output_ORI, map1, map2, map3 = net(inputs)

        outputs_com = output_1 + output_2 + output_3 + output_ORI

        loss = criterion(output_ORI, targets).mean()

        test_loss += loss.item()
        _, predicted = torch.max(output_ORI.data, 1)
        _, predicted_com = torch.max(outputs_com.data, 1)

        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        correct_com += predicted_com.eq(targets.data).cpu().sum()

        if batch_idx % 50 == 0:
            print('Step: %d | Loss: %.3f |Combined Acc: %.3f%% (%d/%d)' % (
            batch_idx, test_loss / (batch_idx + 1),
            100. * float(correct_com) / total, correct_com, total))

    test_acc_en = 100. * float(correct_com) / total
    test_loss = test_loss / (idx + 1)

    return test_acc_en, test_loss




class Features(nn.Module):
    def __init__(self, net_layers):
        super(Features, self).__init__()
        self.net_layer_0 = nn.Sequential(net_layers[0])
        self.net_layer_1 = nn.Sequential(net_layers[1])
        self.net_layer_2 = nn.Sequential(net_layers[2])
        self.net_layer_3 = nn.Sequential(net_layers[3])
        self.net_layer_4 = nn.Sequential(*net_layers[4])
        self.net_layer_5 = nn.Sequential(*net_layers[5])
        self.net_layer_6 = nn.Sequential(*net_layers[6])
        self.net_layer_7 = nn.Sequential(*net_layers[7])


    def forward(self, x):
        x = self.net_layer_0(x)
        x = self.net_layer_1(x)
        x = self.net_layer_2(x)
        x = self.net_layer_3(x)
        x = self.net_layer_4(x)
        x1 = self.net_layer_5(x)
        x2 = self.net_layer_6(x1)
        x3 = self.net_layer_7(x2)
        return x1, x2, x3


class Anti_Noise_Decoder(nn.Module):
    def __init__(self, scale, in_channel):
        super(Anti_Noise_Decoder, self).__init__()
        self.Sigmoid = nn.Sigmoid()

        in_channel = in_channel // (scale * scale)

        self.skip = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1, bias=False),
            nn.LeakyReLU(negative_slope=0.1, inplace=True),
            nn.Conv2d(64, 3, 3, 1, 1, bias=False),
            nn.LeakyReLU(negative_slope=0.1, inplace=True)

        )

        self.process = nn.Sequential(
            nn.PixelShuffle(scale),
            nn.Conv2d(in_channel, 256, 3, 1, 1, bias=False),
            nn.LeakyReLU(negative_slope=0.1, inplace=True),
            nn.PixelShuffle(2),
            nn.Conv2d(64, 128, 3, 1, 1, bias=False),
            nn.LeakyReLU(negative_slope=0.1, inplace=True),
            nn.PixelShuffle(2),
            nn.Conv2d(32, 64, 3, 1, 1, bias=False),
            nn.LeakyReLU(negative_slope=0.1, inplace=True),
            nn.PixelShuffle(2),
            nn.Conv2d(16, 3, 3, 1, 1, bias=False),
            nn.LeakyReLU(negative_slope=0.1, inplace=True)
        )

    def forward(self, x, map):
        return self.skip(x) + self.process(map)

class Network_Wrapper(nn.Module):
    def __init__(self, net_layers, num_class, classifier):
        super().__init__()
        self.Features = Features(net_layers)
        self.classifier_pool = nn.Sequential(classifier[0])
        self.classifier_initial = nn.Sequential(classifier[1])
        self.sigmoid = nn.Sigmoid()
        self.lrelu = nn.LeakyReLU(negative_slope=0.1, inplace=True)

        self.max_pool1 = nn.MaxPool2d(kernel_size=56, stride=1)
        self.max_pool2 = nn.MaxPool2d(kernel_size=28, stride=1)
        self.max_pool3 = nn.MaxPool2d(kernel_size=14, stride=1)

        self.conv_block1 = nn.Sequential(
            BasicConv(512, 512, kernel_size=1, stride=1, padding=0, relu=True),
            BasicConv(512, 1024, kernel_size=3, stride=1, padding=1, relu=True)
        )
        self.classifier1 = nn.Sequential(
            nn.BatchNorm1d(1024),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ELU(inplace=True),
            nn.Linear(512, num_class)
        )

        self.conv_block2 = nn.Sequential(
            BasicConv(1024, 512, kernel_size=1, stride=1, padding=0, relu=True),
            BasicConv(512, 1024, kernel_size=3, stride=1, padding=1, relu=True)
        )
        self.classifier2 = nn.Sequential(
            nn.BatchNorm1d(1024),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ELU(inplace=True),
            nn.Linear(512, num_class),
        )

        self.conv_block3 = nn.Sequential(
            BasicConv(2048, 512, kernel_size=1, stride=1, padding=0, relu=True),
            BasicConv(512, 1024, kernel_size=3, stride=1, padding=1, relu=True)
        )
        self.classifier3 = nn.Sequential(
            nn.BatchNorm1d(1024),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ELU(inplace=True),
            nn.Linear(512, num_class),
        )


    def forward(self, x):
        x1, x2, x3 = self.Features(x)
        map1 = x1.clone()
        map2 = x2.clone()
        map3 = x3.clone()

        classifiers = self.classifier_pool(x3).view(x3.size(0), -1)
        classifiers = self.classifier_initial(classifiers)

        x1_ = self.conv_block1(x1)
        x1_ = self.max_pool1(x1_)
        x1_f = x1_.view(x1_.size(0), -1)
        x1_c = self.classifier1(x1_f)

        x2_ = self.conv_block2(x2)
        x2_ = self.max_pool2(x2_)
        x2_f = x2_.view(x2_.size(0), -1)
        x2_c = self.classifier2(x2_f)


        x3_ = self.conv_block3(x3)
        x3_ = self.max_pool3(x3_)
        x3_f = x3_.view(x3_.size(0), -1)
        x3_c = self.classifier3(x3_f)

        return x1_c, x2_c, x3_c, classifiers, map1, map2, map3

def img_add_noise(x, transformation_seq):

    x = x.permute(0, 2, 3, 1)
    x = x.cpu().numpy()
    x = transformation_seq(images=x)
    x = torch.from_numpy(x.astype(np.float32))
    x = x.permute(0, 3, 1, 2)
    return x


def CELoss(x, y):
    return smooth_crossentropy(x, y, smoothing=0.1)


def train(nb_epoch, batch_size, store_name, num_class=0, start_epoch=0, data_path=''):

    alpha = 1

    exp_dir = store_name
    try:
        os.stat(exp_dir)
    except:
        os.makedirs(exp_dir)

    use_cuda = torch.cuda.is_available()
    print(use_cuda)


    print('==> Preparing data..')
    
    # data preparation
    transform_train = transforms.Compose([
        transforms.Resize((550, 550)),
        transforms.RandomCrop(448, padding=8),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    trainset = torchvision.datasets.ImageFolder(root=data_path+'/train', transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=1)

    
    # Model Initialization
    net = torchvision.models.resnet50()
    state_dict = load_state_dict_from_url('https://download.pytorch.org/models/resnet50-19c8e357.pth')
    net.load_state_dict(state_dict)
    fc_features = net.fc.in_features
    net.fc = nn.Linear(fc_features, num_class)

    net_layers = list(net.children())
    classifier = net_layers[8:10]
    net_layers = net_layers[0:8]
    net = Network_Wrapper(net_layers, num_class, classifier)

    netp = torch.nn.DataParallel(net, device_ids=[0])



    device = torch.device("cuda")
    net.to(device)
    decoder1 = Anti_Noise_Decoder(1, 512).to(device)
    decoder2 = Anti_Noise_Decoder(2, 1024).to(device)
    decoder3 = Anti_Noise_Decoder(4, 2048).to(device)



    CB_loss = CharbonnierLoss()

    base_optimizer = torch.optim.SGD


    optimizer = SAM([
        {'params': net.classifier_initial.parameters(), 'lr': 0.002},
        {'params': net.conv_block1.parameters(), 'lr': 0.002},
        {'params': net.classifier1.parameters(), 'lr': 0.002},
        {'params': net.conv_block2.parameters(), 'lr': 0.002},
        {'params': net.classifier2.parameters(), 'lr': 0.002},
        {'params': net.conv_block3.parameters(), 'lr': 0.002},
        {'params': net.classifier3.parameters(), 'lr': 0.002},

        {'params': decoder1.skip.parameters(), 'lr': 0.002},
        {'params': decoder1.process.parameters(), 'lr': 0.002},
        {'params': decoder2.skip.parameters(), 'lr': 0.002},
        {'params': decoder2.process.parameters(), 'lr': 0.002},
        {'params': decoder3.skip.parameters(), 'lr': 0.002},
        {'params': decoder3.process.parameters(), 'lr': 0.002},

        {'params': net.Features.parameters(), 'lr': 0.0002}

    ],
        base_optimizer, adaptive=False, momentum=0.9, weight_decay=5e-4)



    max_val_acc = 0
    lr = [0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002,  0.002, 0.002,  0.002, 0.002,  0.002, 0.002, 0.0002]
    flag = True
    if start_epoch > 0:
        # Load the checkpoint
        if flag:
            checkpoint = torch.load('/kaggle/input/pmal-checkpoint-model/' + store_name + '/checkpoint.pth')
    
            # Load states into the model and decoders
            net.load_state_dict(checkpoint['model_state_dict'])
            decoder1.load_state_dict(checkpoint['decoder1_state_dict'])
            decoder2.load_state_dict(checkpoint['decoder2_state_dict'])
            decoder3.load_state_dict(checkpoint['decoder3_state_dict'])
    
            # Load metadata
            start_epoch = checkpoint['epoch'] + 1  # Resume from the next epoch
            max_val_acc = checkpoint['max_val_acc']
            flag = False
    if not flag:
        print(type(checkpoint))
        print(type(decoder1))
        print(start_epoch)
        print(max_val_acc)
        
    
        
    for epoch in range(start_epoch, nb_epoch):
        print('\nEpoch: %d' % epoch)
        net.train()
        train_loss = 0
        train_loss1 = 0
        train_loss2 = 0
        train_loss3 = 0
        train_loss4 = 0
        train_loss5 = 0
        correct = 0
        total = 0
        idx = 0
        for batch_idx, (inputs, targets) in enumerate(trainloader):
            idx = batch_idx
            if inputs.shape[0] < batch_size:
                continue
            if use_cuda:
                inputs, targets = inputs.to(device), targets.to(device)
            inputs, targets = Variable(inputs), Variable(targets)


            for nlr in range(len(optimizer.param_groups)):
                optimizer.param_groups[nlr]['lr'] = cosine_anneal_schedule(epoch, nb_epoch, lr[nlr])

            sometimes_1 = lambda aug: iaa.Sometimes(0.2, aug)
            sometimes_2 = lambda aug: iaa.Sometimes(0.5, aug)



            trans_seq_aug = iaa.Sequential(
                [

                    sometimes_1(iaa.Affine(
                        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                        rotate=(-15, 15),
                        shear=(-15, 15),
                        order=[0, 1],
                        cval=(0, 1),
                        mode=ia.ALL
                    )),
                    sometimes_2(iaa.GaussianBlur((0, 3.0)))
                ],
                random_order=True
            )

            trans_seq = iaa.Sequential(
                [

                    iaa.AdditiveGaussianNoise(
                        loc=0, scale=(0.0, 0.05), per_channel=0.5
                    )
                ],
                random_order=True
            )

            # H1
            # H1 first forward-backward step
            enable_running_stats(netp)
            optimizer.zero_grad()
            inputs1_gt = img_add_noise(inputs, trans_seq_aug).to(device)
            inputs1 = img_add_noise(inputs1_gt, trans_seq).to(device)
            output_1, _, _, _, map1, _, _ = netp(inputs1)
            loss1_c = CELoss(output_1, targets).mean() * 1

            inputs1_syn = decoder1(inputs1, map1)
            loss1_g = CB_loss(inputs1_syn, inputs1_gt) * 1

            output_1_syn, _, _, _, _, _, _ = netp(inputs1_syn)
            loss1_c_syn = CELoss(output_1_syn, targets).mean() * 1

            loss1 = loss1_c + (alpha * loss1_g) + loss1_c_syn
            loss1.backward()
            optimizer.first_step(zero_grad=True)


            # H1 second forward-backward step
            disable_running_stats(netp)
            optimizer.zero_grad()

            output_1, _, _, _, map1, _, _ = netp(inputs1)
            loss1_c = CELoss(output_1, targets).mean() * 1

            inputs1_syn = decoder1(inputs1, map1)
            loss1_g = CB_loss(inputs1_syn, inputs1_gt) * 1

            output_1_syn, _, _, _, _, _, _ = netp(inputs1_syn)
            loss1_c_syn = CELoss(output_1_syn, targets).mean() * 1

            loss1_ = loss1_c + (alpha * loss1_g) + loss1_c_syn
            loss1_.backward()
            optimizer.second_step(zero_grad=True)


            # H2
            # H2 first forward-backward step
            enable_running_stats(netp)
            optimizer.zero_grad()
            inputs2_gt = img_add_noise(inputs, trans_seq_aug).to(device)
            inputs2 = img_add_noise(inputs2_gt, trans_seq).to(device)
            _, output_2, _, _, _, map2, _ = netp(inputs2)
            loss2_c = CELoss(output_2, targets).mean() * 1

            inputs2_syn = decoder2(inputs2, map2)
            loss2_g = CB_loss(inputs2_syn, inputs2_gt) * 1

            _, output_2_syn, _, _, _, _, _ = netp(inputs2_syn)
            loss2_c_syn = CELoss(output_2_syn, targets).mean() * 1

            loss2 = loss2_c + (alpha * loss2_g) + loss2_c_syn
            loss2.backward()
            optimizer.first_step(zero_grad=True)

            # H2 second forward-backward step
            disable_running_stats(netp)
            optimizer.zero_grad()
            _, output_2, _, _, _, map2, _ = netp(inputs2)
            loss2_c = CELoss(output_2, targets).mean() * 1

            inputs2_syn = decoder2(inputs2, map2)
            loss2_g = CB_loss(inputs2_syn, inputs2_gt) * 1

            _, output_2_syn, _, _, _, _, _ = netp(inputs2_syn)
            loss2_c_syn = CELoss(output_2_syn, targets).mean() * 1

            loss2_ = loss2_c + (alpha * loss2_g) + loss2_c_syn
            loss2_.backward()
            optimizer.second_step(zero_grad=True)

            #H3
            # H3 first forward-backward step
            enable_running_stats(netp)
            optimizer.zero_grad()
            inputs3_gt = img_add_noise(inputs, trans_seq_aug).to(device)
            inputs3 = img_add_noise(inputs3_gt, trans_seq).to(device)
            _, _, output_3, _, _, _, map3 = netp(inputs3)
            loss3_c = CELoss(output_3, targets).mean() * 1

            inputs3_syn = decoder3(inputs3, map3)
            loss3_g = CB_loss(inputs3_syn, inputs3_gt) * 1

            _, _, output_3_syn, _, _, _, _ = netp(inputs3_syn)
            loss3_c_syn = CELoss(output_3_syn, targets).mean() * 1

            loss3 = loss3_c + (alpha * loss3_g) + loss3_c_syn
            loss3.backward()
            optimizer.first_step(zero_grad=True)

            # H3 second forward-backward step
            disable_running_stats(netp)
            optimizer.zero_grad()
            _, _, output_3, _, _, _, map3 = netp(inputs3)
            loss3_c = CELoss(output_3, targets).mean() * 1

            inputs3_syn = decoder3(inputs3, map3)
            loss3_g = CB_loss(inputs3_syn, inputs3_gt) * 1

            _, _, output_3_syn, _, _, _, _ = netp(inputs3_syn)
            loss3_c_syn = CELoss(output_3_syn, targets).mean() * 1

            loss3_ = loss3_c + (alpha * loss3_g) + loss3_c_syn
            loss3_.backward()
            optimizer.second_step(zero_grad=True)



            # H4
            # H4 first forward-backward step
            enable_running_stats(netp)
            optimizer.zero_grad()
            output_1_final, output_2_final, output_3_final, output_ORI, _, _, _ = netp(inputs)
            ORI_loss = CELoss(output_1_final, targets).mean() + \
                                CELoss(output_2_final, targets).mean() + \
                                CELoss(output_3_final, targets).mean() + \
                                CELoss(output_ORI, targets).mean() * 2
            ORI_loss.backward()
            optimizer.first_step(zero_grad=True)

            # H4 second forward-backward step
            disable_running_stats(netp)
            optimizer.zero_grad()
            output_1_final, output_2_final, output_3_final, output_ORI, _, _, _ = netp(inputs)
            ORI_loss_ = CELoss(output_1_final, targets).mean() + \
                          CELoss(output_2_final, targets).mean() + \
                          CELoss(output_3_final, targets).mean() + \
                          CELoss(output_ORI, targets).mean() * 2
            ORI_loss_.backward()
            optimizer.second_step(zero_grad=True)


            _, predicted = torch.max(output_ORI.data, 1)
            total += targets.size(0)
            correct += predicted.eq(targets.data).cpu().sum()

            train_loss += (loss1.item() + loss2.item() + loss3.item() + ORI_loss.item())
            train_loss1 += loss1.item()
            train_loss2 += loss2.item()
            train_loss3 += loss3.item()
            train_loss4 += (loss1_g.item() + loss2_g.item() + loss3_g.item())
            train_loss5 += ORI_loss.item()

            if batch_idx % 50 == 0:
                print(
                    'Step: %d | Loss1: %.3f | Loss2: %.5f | Loss3: %.5f | Loss_Gen: %.5f |Loss_ORI: %.5f | Loss: %.3f | Acc: %.3f%% (%d/%d)' % (
                    batch_idx, train_loss1 / (batch_idx + 1), train_loss2 / (batch_idx + 1),
                    train_loss3 / (batch_idx + 1), train_loss4 / (batch_idx + 1),  train_loss5/ (batch_idx + 1), train_loss / (batch_idx + 1),
                    100. * float(correct) / total, correct, total))

        train_acc = 100. * float(correct) / total
        train_loss = train_loss / (idx + 1)
        with open(exp_dir + '/results_train.txt', 'a') as file:
            file.write(
                'Iteration %d | train_acc = %.5f | train_loss = %.5f | Loss1: %.3f | Loss2: %.5f | Loss3: %.5f | Loss_Gen: %.5f | Loss_ORI: %.5f |\n' % (
                epoch, train_acc, train_loss, train_loss1 / (idx + 1), train_loss2 / (idx + 1), train_loss3 / (idx + 1),
                train_loss4 / (idx + 1), train_loss5 / (idx + 1)))


        val_acc_com, val_loss = test(net, CELoss, 7, data_path+'/test')
        if val_acc_com > max_val_acc:
            max_val_acc = val_acc_com
            net.cpu()
            decoder1.cpu()
            decoder2.cpu()
            decoder3.cpu()
            torch.save(net, './' + store_name + '/model.pth')
            torch.save(decoder1, './' + store_name + '/decoder1.pth')
            torch.save(decoder2, './' + store_name + '/decoder2.pth')
            torch.save(decoder3, './' + store_name + '/decoder3.pth')
            net.to(device)
            decoder1.to(device)
            decoder2.to(device)
            decoder3.to(device)
        with open(exp_dir + '/results_test.txt', 'a') as file:
            file.write('Iteration %d, test_acc_combined = %.5f, test_loss = %.6f\n' % (
            epoch, val_acc_com, val_loss))
        checkpoint = {
            'epoch': epoch,
            'max_val_acc': max_val_acc,
            'model_state_dict': net.state_dict(),
            'decoder1_state_dict': decoder1.state_dict(),
            'decoder2_state_dict': decoder2.state_dict(),
            'decoder3_state_dict': decoder3.state_dict(),
        }

        # Save the checkpoint
        torch.save(checkpoint, './' + store_name + '/checkpoint.pth')
    

if __name__ == '__main__':
    data_path = '/kaggle/input/anti-noise-fgvr-v2/dataset_folder'
    if not os.path.isdir('results'):
        os.mkdir('results')
    train(nb_epoch=50,             # number of epoch
             batch_size=8,         # batch size
             store_name='results/Stanford_Cars_ResNet50_PMAL',     # the folder for saving results
             num_class=5,          # number of categories
             start_epoch=25,         # the start epoch number
             data_path = data_path)   # the path to the dataset



True
==> Preparing data..


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 544MB/s]
/tmp/ipykernel_23/491003624.py:312: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loade

<class 'dict'>
<class '__main__.Anti_Noise_Decoder'>
25
100.0

Epoch: 25
Step: 0 | Loss1: 0.209 | Loss2: 0.18278 | Loss3: 0.09784 | Loss_Gen: 0.11495 |Loss_ORI: 0.26104 | Loss: 0.750 | Acc: 100.000% (8/8)
Step: 50 | Loss1: 0.250 | Loss2: 0.13021 | Loss3: 0.09568 | Loss_Gen: 0.15285 |Loss_ORI: 0.20426 | Loss: 0.681 | Acc: 98.775% (403/408)
Step: 100 | Loss1: 0.249 | Loss2: 0.12074 | Loss3: 0.10584 | Loss_Gen: 0.14913 |Loss_ORI: 0.20909 | Loss: 0.685 | Acc: 98.886% (799/808)
Step: 150 | Loss1: 0.252 | Loss2: 0.11478 | Loss3: 0.10293 | Loss_Gen: 0.15061 |Loss_ORI: 0.21087 | Loss: 0.680 | Acc: 98.013% (1184/1208)
Step: 200 | Loss1: 0.250 | Loss2: 0.11119 | Loss3: 0.10501 | Loss_Gen: 0.15045 |Loss_ORI: 0.21012 | Loss: 0.676 | Acc: 97.948% (1575/1608)
Step: 250 | Loss1: 0.246 | Loss2: 0.10810 | Loss3: 0.09994 | Loss_Gen: 0.15047 |Loss_ORI: 0.20282 | Loss: 0.657 | Acc: 97.958% (1967/2008)
Step: 300 | Loss1: 0.244 | Loss2: 0.10479 | Loss3: 0.09814 | Loss_Gen: 0.15044 |Loss_ORI: 0.20021 | Loss:

/tmp/ipykernel_23/491003624.py:49: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  inputs, targets = Variable(inputs, volatile=True), Variable(targets)


Step: 0 | Loss: 0.036 |Combined Acc: 100.000% (7/7)
Step: 50 | Loss: 0.062 |Combined Acc: 100.000% (357/357)
Step: 100 | Loss: 0.058 |Combined Acc: 100.000% (707/707)

Epoch: 26
Step: 0 | Loss1: 0.215 | Loss2: 0.06237 | Loss3: 0.05075 | Loss_Gen: 0.17560 |Loss_ORI: 0.10759 | Loss: 0.436 | Acc: 100.000% (8/8)
Step: 50 | Loss1: 0.221 | Loss2: 0.09083 | Loss3: 0.09455 | Loss_Gen: 0.14633 |Loss_ORI: 0.19385 | Loss: 0.600 | Acc: 98.775% (403/408)
Step: 100 | Loss1: 0.233 | Loss2: 0.10266 | Loss3: 0.10276 | Loss_Gen: 0.14622 |Loss_ORI: 0.20904 | Loss: 0.648 | Acc: 98.144% (793/808)
Step: 150 | Loss1: 0.236 | Loss2: 0.09909 | Loss3: 0.09908 | Loss_Gen: 0.14767 |Loss_ORI: 0.20464 | Loss: 0.639 | Acc: 98.427% (1189/1208)
Step: 200 | Loss1: 0.234 | Loss2: 0.09839 | Loss3: 0.09686 | Loss_Gen: 0.14840 |Loss_ORI: 0.19914 | Loss: 0.629 | Acc: 98.507% (1584/1608)
Step: 250 | Loss1: 0.233 | Loss2: 0.09899 | Loss3: 0.09417 | Loss_Gen: 0.14896 |Loss_ORI: 0.19692 | Loss: 0.623 | Acc: 98.406% (1976/2008)
